In [1]:
# from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Convolution2D, Conv2D, MaxPool2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
import matplotlib.pyplot as plt

In [3]:
TRAIN_PATH = './data/test/DataSet_SEQUENCE_3' # 1 is for training
TEST_PATH = './data/test/DataSet_SEQUENCE_2' # 1 is for training
MODEL_PATH = './data/saved_model'
IMAGE_DIM = (640, 480)
BATCH = 10
WORKERS = 10

def directory_iterator(directory):
    return image_dataset_from_directory(
        directory,
        image_size=IMAGE_DIM,
        batch_size=BATCH,
        labels='inferred',
        label_mode='categorical'
    )

train_dataset = directory_iterator(TRAIN_PATH)
test_dataset = directory_iterator(TEST_PATH)

Found 1800 files belonging to 9 classes.
Found 1800 files belonging to 9 classes.


In [ ]:
train_dataset

In [4]:
INPUT_DIM = IMAGE_DIM + (3,)

In [5]:
# Simple CNN 
# (1): input_shape in 1st layer must match image dimension
# (2): output in last layer must match one-hot representation of labels

model= Sequential()
model.add(Conv2D(kernel_size=(3,3), filters=32, activation='tanh', input_shape=INPUT_DIM))
model.add(Conv2D(filters=30,kernel_size = (3,3),activation='tanh'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(filters=30,kernel_size = (3,3),activation='tanh'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(filters=30,kernel_size = (3,3),activation='tanh'))

model.add(Flatten())

model.add(Dense(20,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(9,activation = 'softmax'))

In [ ]:
# # Build CNN
# # VGG-16 clone, doesn't work because last layers have wrong dim

# model = Sequential()
# model.add(ZeroPadding2D((1,1),input_shape=INPUT_DIM))
# model.add(Convolution2D(64, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(Convolution2D(4096, (7, 7), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Convolution2D(4096, (1, 1), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Convolution2D(2622, (1, 1)))
# model.add(Flatten())
# model.add(Activation('softmax'))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [25]:
model.fit(train_dataset, epochs=2, workers=WORKERS, use_multiprocessing=True)

Epoch 1/2
180/180 [==============================] - 424s 2s/step - loss: 2.1977 - accuracy: 0.1050
Epoch 2/2
180/180 [==============================] - 433s 2s/step - loss: 2.1977 - accuracy: 0.1033


In [9]:
import time
CHECKPOINT_PATH = MODEL_PATH + '_' + str(int(time.time()))
model.save(CHECKPOINT_PATH)

INFO:tensorflow:Assets written to: ./data/saved_model_1615183531/assets


In [ ]:
model = load_model(MODEL_PATH)

In [ ]:
model.summary()

In [10]:
from tensorflow.compat.v1.metrics import average_precision_at_k

In [26]:
predictions = model.predict(test_dataset, workers=WORKERS, use_multiprocessing=True)
predictions

array([[0.11120996, 0.11125054, 0.11115114, ..., 0.10976741, 0.11153789,
        0.11175682],
       [0.11120996, 0.11125054, 0.11115114, ..., 0.10976741, 0.11153789,
        0.11175682],
       [0.11120996, 0.11125054, 0.11115114, ..., 0.10976741, 0.11153789,
        0.11175682],
       ...,
       [0.11120996, 0.11125054, 0.11115114, ..., 0.10976741, 0.11153789,
        0.11175682],
       [0.11120996, 0.11125054, 0.11115114, ..., 0.10976741, 0.11153789,
        0.11175682],
       [0.11120996, 0.11125054, 0.11115114, ..., 0.10976741, 0.11153789,
        0.11175682]], dtype=float32)

In [14]:
predictions.shape

(1800, 9)

In [19]:
(predictions.max(axis=1) - predictions.min(axis=1)).max()

0.003055632

In [27]:
predicted_labels = predictions.argmax(axis=1)

In [28]:
np.unique(predicted_labels)

array([8])

In [24]:
model.evaluate(test_dataset)

180/180 [==============================] - 86s 462ms/step - loss: 2.1973 - accuracy: 0.1111


[2.197258472442627, 0.1111111119389534]

In [ ]:
# TODO: compute mAP
    
# labels = None # test_dataset labels
# N = predictions.shape[0] # test_dataset size

In [ ]:
# ap_at_N = average_precision_at_k(labels=labels, predictions=predictions, k=N)
# ap_at_N

## ########################################################

In [44]:
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=IMAGE_DIM,
    batch_size=BATCH,
    labels='inferred',
    label_mode='categorical'
)
val_ds = image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=IMAGE_DIM,
    batch_size=BATCH,
    labels='inferred',
    label_mode='categorical'
)

Found 1800 files belonging to 9 classes.
Using 1440 files for training.
Found 1800 files belonging to 9 classes.
Using 360 files for validation.


In [36]:
import keras
from keras import layers


def get_augmentation():
    data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ])
    return data_augmentation


def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    data_augmentation = get_augmentation()
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [45]:
train_ds.class_names

['Corridor1_RGB',
 'Corridor2_RGB',
 'Corridor3_RGB',
 'D3A_RGB',
 'D7_RGB',
 'F102_RGB',
 'F104_RGB',
 'F105_RGB',
 'F107_RGB']

In [46]:
model = make_model(INPUT_DIM, len(train_ds.class_names))

In [40]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 640, 480, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 640, 480, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 640, 480, 3)  0           sequential_1[0][0]               
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 320, 240, 32) 896         rescaling[0][0]                  
______________________________________________________________________________________________

In [47]:
epochs = 10

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)

Epoch 1/10
144/144 [==============================] - 734s 5s/step - loss: 1.5137 - accuracy: 0.4933 - val_loss: 2.3634 - val_accuracy: 0.1222
Epoch 2/10
144/144 [==============================] - 1074s 7s/step - loss: 0.7933 - accuracy: 0.7083 - val_loss: 2.7745 - val_accuracy: 0.1917
Epoch 3/10
144/144 [==============================] - 741s 5s/step - loss: 0.6100 - accuracy: 0.7707 - val_loss: 2.4833 - val_accuracy: 0.3917
Epoch 4/10
144/144 [==============================] - 747s 5s/step - loss: 0.4590 - accuracy: 0.8327 - val_loss: 2.6827 - val_accuracy: 0.2833
Epoch 5/10
144/144 [==============================] - 721s 5s/step - loss: 0.5248 - accuracy: 0.8070 - val_loss: 4.6871 - val_accuracy: 0.4028
Epoch 6/10
144/144 [==============================] - 720s 5s/step - loss: 0.4765 - accuracy: 0.8286 - val_loss: 0.9617 - val_accuracy: 0.6611
Epoch 7/10
144/144 [==============================] - 723s 5s/step - loss: 0.4741 - accuracy: 0.8319 - val_loss: 0.3580 - val_accuracy: 0.844

In [48]:
preds = model.predict(test_dataset)

In [49]:
preds.shape

(1800, 9)

In [50]:
np.unique(preds.argmax(axis=1))

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [51]:
import time
CHECKPOINT_PATH = MODEL_PATH + '_' + str(int(time.time()))
model.save(CHECKPOINT_PATH)

INFO:tensorflow:Assets written to: ./data/saved_model_1615292750/assets
